# Black-Scholes Model Analysis

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [12]:
import numpy as np
from scipy.stats import norm
from scipy import optimize
import sympy as sp


# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description: The Black-Scholes Equation

The Black-Scholes model is a cornerstone of modern financial theory, introduced by Fischer Black and Myron Scholes in 1973. It provides a mathematical model for pricing European-style options, which are financial derivatives allowing for the purchase or sale of an underlying asset at a predetermined price on a specific date. The model's significance lies in its ability to deduce a theoretical estimate of an option's price in the market, absent of any arbitrage opportunities (i.e., risk-free profit).

### Purpose of the Model

The primary purpose of the Black-Scholes model is to determine the price of options, specifically European call and put options, under the assumption of constant volatility and interest rates. By doing so, it helps traders, financial analysts, and investors to:

- **Evaluate Options:** Determine fair prices for options, which are otherwise difficult due to the option's dependency on future asset prices.
- **Risk Management:** Understand how different factors affect the option's value, aiding in the formulation of strategies to hedge against price movements of the underlying asset.
- **Investment Decision Making:** Make informed decisions regarding the purchase or sale of options based on their theoretical values.

### The Black-Scholes Formula

The Black-Scholes model for a European call option's price, \(C\), and a European put option's price, \(P\), on a non-dividend-paying stock is given by:

#### For a Call Option:
$$C(S, t) = S_0N(d_1) - Ke^{-rT}N(d_2)$$

#### For a Put Option:
$$P(S, t) = Ke^{-rT}N(-d_2) - S_0N(-d_1) $$

where:
- $S_0$ is the current price of the stock.
- $K$ is the strike price of the option.
- $r$ is the risk-free interest rate (annual).
- $T$ is the time to expiration of the option (in years).
- $N (\cdot)$ represents the cumulative distribution function of the standard normal distribution.
- $d_1$ and $d_2$ are calculated as follows:

$$d_1 = \frac{\ln(\frac{S_0}{K}) + (r + \frac{\sigma^2}{2})T}{\sigma\sqrt{T}} $$

$$d_2 = d_1 - \sigma\sqrt{T} $$

- $\sigma$ is the volatility of the stock's returns (standard deviation of the stock's returns).

### Understanding the Equations

- **$S_0N(d_1)$** and **$-S_0N(-d_1)$**: These terms represent the present value of purchasing the stock outright, adjusted for the probability that the option finishes in the money (i.e., it has intrinsic value at expiration).
- **$Ke^{-rT}N(d_2)$** and **$Ke^{-rT}N(-d_2)$**: These terms account for the present value of paying the strike price at expiration, adjusted for the probability the option will be exercised.

The function $N(d_1)$ and $N(d_2)$ are probabilities derived from the cumulative distribution of a standard normal distribution, which account for the risk-adjusted probabilities that the option will be in the money at expiration.


The Black-Scholes model revolutionized financial markets by providing a systematic method to price European options. Its derivation assumes market efficiency, no arbitrage opportunities, and constant volatility and interest rates. While real-world conditions may deviate from these assumptions, the Black-Scholes model remains a fundamental tool in finance for theoretical valuation and risk management strategies.


## Analytical Solution 

The Black-Scholes model provides a closed-form solution for pricing European options, a pivotal concept in financial mathematics. This solution is derived under the framework of certain assumptions, including constant volatility and no dividends. Below, we explore the derivation and application of this formula for a European call option, which can be extended similarly for a put option.

### Purpose and Significance

The Black-Scholes formula allows investors and financial analysts to determine the theoretical price of European call and put options. Understanding this formula and its dependencies helps in making informed decisions in financial markets.

### Derivation of the Black-Scholes Formula

The formula for a European call option is as follows, where the option's price is a function of the stock price, strike price, risk-free rate, volatility, and time to expiration.

#### For a Call Option:

The price $C$ is given by:
$$ C(S, t) = S_0N(d_1) - Ke^{-rT}N(d_2) $$

### Characterizing the Option's Price as a Function of Volatility

Volatility $(\sigma)$ plays a crucial role in option pricing, influencing the option's sensitivity to changes in the underlying asset's price. Below, we characterize the call option's price as a function of volatility, demonstrating the impact of volatility on pricing.


In [14]:
from modelproject import BlackScholesModel

# Initialize the model with specific parameters
model = BlackScholesModel(S=100, K=100, T=1, r=0.05, sigma=0.2)

# Calculate the call option price
call_price = model.call_price()
print(f"The price of the European call option is: {call_price}")


The price of the European call option is: 10.450583572185565


## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 